In [1]:
%matplotlib inline

import datetime

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import xgboost as xgb

## A: Start

In [2]:
print("start")
df_train = pd.read_pickle('/mnt/sframe/df_train')
df_test = pd.read_pickle('/mnt/sframe/df_test')

df_train['text'].fillna('', inplace=True)
df_test['text'].fillna('', inplace=True)

start


### B.1: test

In [ ]:
print("split")
train_train, train_cv = train_test_split(df_train, test_size=0.2, random_state=107)

TRAIN = train_train
CV = train_cv

### B.2: deploy

In [3]:
TRAIN = df_train
CV = df_test

## C: Model

In [4]:
print("BOW")
count_vect = CountVectorizer(stop_words='english', min_df=20)
TRAIN_counts = count_vect.fit_transform(TRAIN['text'])
CV_counts = count_vect.transform(CV['text'])
print("shape TRAIN_counts: %s" % (str(TRAIN_counts.shape)))

print("TF-IDF")
tfidf_transformer = TfidfTransformer()
TRAIN_tfidf = tfidf_transformer.fit_transform(TRAIN_counts)
CV_tfidf = tfidf_transformer.transform(CV_counts)
print("shape TRAIN_tfidf: %s" % (str(TRAIN_tfidf.shape)))

BOW
shape TRAIN_counts: (337024, 108062)
TF-IDF
shape TRAIN_tfidf: (337024, 108062)


In [5]:
print("Combine features")
bad_cols = [col for col in list(TRAIN.columns) 
            if col.startswith('word2vec') or col in set(['text_words', 'text', 'id', 'sponsored'])]

DF_TRAIN = TRAIN.drop(bad_cols, 1).fillna(0)
DF_CV = CV.drop(bad_cols, 1).fillna(0)

DF_TRAIN = DF_TRAIN.astype('float64')
DF_CV = DF_CV.astype('float64')

DF_DF_TRAIN = sp.hstack( [TRAIN_tfidf,  sp.csr_matrix(np.array(DF_TRAIN)) ] )
DF_DF_CV = sp.hstack( [CV_tfidf,  sp.csr_matrix(np.array(DF_CV)) ] )

print("shape DF_TRAIN: %s" % (str(DF_TRAIN.shape)))

Combine features
shape DF_TRAIN: (337024, 50)


In [6]:
print("shape DF_DF_TRAIN: %s" % (str(DF_DF_TRAIN.shape)))

shape DF_DF_TRAIN: (337024, 108112)


## D: Combine

### D.1: test

In [ ]:
dtrain = xgb.DMatrix(DF_DF_TRAIN, label = TRAIN['sponsored'])
dtest = xgb.DMatrix(DF_DF_CV, label = CV['sponsored'])
watchlist  = [(dtest,'eval'), (dtrain,'train')]

### D.2: deploy

In [8]:
dtrain = xgb.DMatrix(DF_DF_TRAIN, label = TRAIN['sponsored'])
dtest = xgb.DMatrix(DF_DF_CV)
watchlist  = [(dtrain,'train')]

In [9]:
# Save for later:
dtrain.save_binary('orig_dtrain.buffer_mindf20')
dtest.save_binary('orig_dtest.buffer_mindf20')

## E: Params

In [10]:
#max depth [8], eta 0.2, colsample_bytree 0.5, subsample: 1.0
param = {'max_depth': 17, 
         'eta': 0.05, 
         #'gamma': 1.0,
         'colsample_bytree': 0.6,
         'subsample': 1.0,
         'min_child_weight': 7,
         'silent':1, 
         'objective':'binary:logistic', 
         'eval_metric':'auc',
         'early_stopping_rounds':20}
num_round = 800

print(datetime.datetime.now())

2015-10-06 16:41:57.080669


In [11]:
bst = xgb.train(param, dtrain, num_round, watchlist)

[0]	train-auc:0.750854
[1]	train-auc:0.803894
[2]	train-auc:0.809665
[3]	train-auc:0.818385
[4]	train-auc:0.827795
[5]	train-auc:0.830302
[6]	train-auc:0.834035
[7]	train-auc:0.839013
[8]	train-auc:0.844391
[9]	train-auc:0.848326
[10]	train-auc:0.851132
[11]	train-auc:0.852845
[12]	train-auc:0.854122
[13]	train-auc:0.857297
[14]	train-auc:0.860236
[15]	train-auc:0.866022
[16]	train-auc:0.871069
[17]	train-auc:0.874039
[18]	train-auc:0.878415
[19]	train-auc:0.885498
[20]	train-auc:0.891266
[21]	train-auc:0.893285
[22]	train-auc:0.898215
[23]	train-auc:0.900397
[24]	train-auc:0.906033
[25]	train-auc:0.911527
[26]	train-auc:0.914070
[27]	train-auc:0.915713
[28]	train-auc:0.918840
[29]	train-auc:0.920841
[30]	train-auc:0.924216
[31]	train-auc:0.927562
[32]	train-auc:0.929592
[33]	train-auc:0.931161
[34]	train-auc:0.932303
[35]	train-auc:0.934801
[36]	train-auc:0.936846
[37]	train-auc:0.937568
[38]	train-auc:0.939899
[39]	train-auc:0.941443
[40]	train-auc:0.943318
[41]	train-auc:0.944972
[4

In [12]:
print(datetime.datetime.now())
preds = bst.predict(dtest) #, ntree_limit=bst.boost .best_iteration)

2015-10-06 17:22:42.101027


In [ ]:
roc_auc_score(CV['sponsored'], preds)

# M-version

In [14]:
#max depth [8], eta 0.2, colsample_bytree 0.5, subsample: 1.0
param = {'max_depth': 18, 
         'eta': 0.02, 
         #'gamma': 1.0,
         'colsample_bytree': 0.6,
         'subsample': 1.0,
         'min_child_weight': 7,
         'silent':1, 
         'objective':'binary:logistic', 
         'eval_metric':'auc',
         'early_stopping_rounds':20}
num_round = 1800

print(datetime.datetime.now())

2015-10-06 17:27:47.087603


In [ ]:
bst2 = xgb.train(param, dtrain, num_round, watchlist)

[0]	train-auc:0.753641
[1]	train-auc:0.806111
[2]	train-auc:0.813202
[3]	train-auc:0.818390
[4]	train-auc:0.820711
[5]	train-auc:0.823589
[6]	train-auc:0.826262
[7]	train-auc:0.832258


In [ ]:
print(datetime.datetime.now())
preds = bst2.predict(dtest) #, ntree_limit=bst.boost .best_iteration)

# S

In [ ]:
submission = pd.DataFrame({
        'file': df_test['id'].map(lambda s: s + '_raw_html.txt').reset_index(drop=True),
        'sponsored': preds
    })
submission.to_csv('submission_xgb_Oct_06_02.csv', index=False)
print(datetime.datetime.now())

In [ ]:
#submission = df_test[['id']].map(lambda s: s + '_raw_html.txt'). reset_index(drop=True)
#submission['sponsored'] = clf.predict_proba(test.drop(['file', 'sponsored'], 1))[:, 1]
#submission.to_csv('submission_xgb_Oct_06_01.csv', index=False)

# T

In [ ]:
#max depth [8], eta 0.2, colsample_bytree 0.5, subsample: 1.0
param = {'max_depth': 16, 
         'eta': 0.1, 
         #'gamma': 1.0,
         'colsample_bytree': 0.5,
         'subsample': 1.0,
         'min_child_weight': 5,
         'silent':1, 
         'objective':'binary:logistic', 
         'eval_metric':'auc',
         'early_stopping_rounds':20}
num_round = 300

print(datetime.datetime.now())

In [ ]:
dtrain.save_binary('dtrain.buffer_mindf20')
dtest.save_binary('dtest.buffer_mindf20')
#a = xgb.DMatrix('dtest.buffer')

In [ ]:
submission = test[['file']].reset_index(drop=True)
submission['sponsored'] = clf.predict_proba(test.drop(['file', 'sponsored'], 1))[:, 1]